# LSTM Clásico

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import json
import matplotlib.pyplot as plt

import os

In [ ]:
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.15.0


In [ ]:
# Replace 'your_folder_path' with the actual path to your folder containing text files
folder_path = './drive/MyDrive/DatosInvestigacion/resultados/energy/data-continua/'

In [ ]:
file_path = os.path.join(folder_path, 'df_clean_all_2_22.csv')
df = pd.read_csv(file_path, index_col=0, parse_dates=True)

In [ ]:
df = df.resample('2T').asfreq()

In [ ]:
df.isna().sum()

Potencia_3F_[W]           0
Potencia_3F_[VA]          0
Factor de Potencia        0
Voltaje_(R)_[V]           0
Voltaje_(S)_[V]           0
Voltaje_(T)_[V]           0
Voltaje_(RS)_[V]          0
Voltaje_(ST)_[V]          0
Voltaje_(TR)_[V]          0
Corriente_R_[A]           0
Corriente_S_[A]           0
Corriente_T_[A]           0
Potencia_R_[VA]           0
Potencia_S_[VA]           0
Potencia_T_[VA]           0
Potencia_R_[W]            0
Potencia_S_[W]            0
Potencia_T_[W]            0
Potencia_R_[VAR]          0
Potencia_S_[VAR]          0
Potencia_T_[VAR]          0
Corriente_N_[A]           0
Frecuencia_[Hz]           0
Potencia_3F_[VAR]         0
suma_potencias_W          0
suma_potencias_VA         0
factor_total_calculado    0
dtype: int64

In [ ]:
df.sort_index(ascending=True, inplace=True)

In [ ]:
train_size = int(len(df)*0.7)
train_size

7367

In [ ]:
val_size = int(len(df)*0.1)
val_size

1052

In [ ]:
df_train = df.iloc[:train_size]

In [ ]:
df_val = df.iloc[train_size:train_size+val_size]

In [ ]:
len(df_val)

1052

In [ ]:
df_test = df.iloc[train_size+val_size:]

In [ ]:
df_train.head(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_T_[W],Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado
2020-12-17 09:10:00,764.273,870.925,0.877,126.198,125.841,126.782,217.997,219.529,218.545,1.436,...,175.091,-3.085,88.859,-68.702,2.926,59.983,17.073,764.273,870.925,0.877542


In [ ]:
df_train.tail(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_T_[W],Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado
2020-12-27 14:42:00,467.076,548.9,0.851,126.404,126.987,128.683,218.673,222.113,220.934,1.282,...,87.546,-5.309,63.17,-38.944,2.004,59.986,18.917,467.076,548.9,0.850931


In [ ]:
df_val.head(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_T_[W],Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado
2020-12-27 14:44:00,453.005,532.047,0.826,126.302,127.01,128.644,218.619,222.142,220.757,1.278,...,84.923,-5.111,61.525,-37.721,1.941,58.233,18.693,464.086,548.099,0.846719


In [ ]:
df_val.tail(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_T_[W],Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado
2020-12-29 01:46:00,498.132,598.352,0.833,128.112,127.41,128.896,220.623,222.668,222.499,1.047,...,222.534,-22.808,-24.12,-122.265,1.562,59.99,-169.192,498.133,598.351,0.83251


In [ ]:
df_test.head(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_T_[W],Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado
2020-12-29 01:48:00,496.704,596.44,0.833,127.926,127.335,128.872,220.47,222.57,222.252,1.047,...,221.716,-22.894,-24.118,-122.223,1.555,59.988,-169.236,496.704,596.441,0.83278


In [ ]:
df_test.tail(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_T_[W],Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado
2020-12-31 23:58:00,702.121,811.937,0.839,127.734,128.241,129.285,220.938,223.671,222.634,1.07,...,344.124,-21.609,60.951,-154.166,1.584,58.249,-114.823,712.963,836.548,0.852268


In [ ]:
input_sets={
    '3F_W':['Potencia_3F_[W]','Corriente_T_[A]','Potencia_T_[VA]'],
    '3F_VA':['Potencia_3F_[VA]','Corriente_T_[A]','Potencia_T_[VA]'],
    '3F_factor':['Factor de Potencia','Potencia_R_[W]']
}

In [ ]:
input_sets.keys()

dict_keys(['3F_W', '3F_VA', '3F_factor'])

In [ ]:
ventana_prediccion = [30,15,5,1]
factor_ventana_entrada = [0.5,1,2]
num_neuronas_set = [50,100,150]
batch_size_set = [50,100,150]
num_layers_set = [1,2,3]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation
from tensorflow import keras
import tensorflow as tf

In [ ]:
def create_Model(n_input, LSTM_neurons = 100, layers=3,n_features=0,n_output=1):
  # define model
  model = Sequential()
  if(layers>1):
    model.add(LSTM(LSTM_neurons, activation='relu', input_shape=(n_input, n_features+1),return_sequences=True))
    for i in range(layers-2):
        model.add(LSTM(LSTM_neurons, activation='relu',return_sequences=True))
    model.add(LSTM(LSTM_neurons, activation='relu'))
  else:
    model.add(LSTM(LSTM_neurons, activation='relu', input_shape=(n_input, n_features+1)))
  model.add(Dense(LSTM_neurons//2, activation='relu'))
  model.add(Dense(n_output))
  model.compile(optimizer='adam', loss='mse')
  return model

In [ ]:
def get_XY(scaled_data,past_hours,prediction_window):
  # Prepare the dataset with lagged features
  X, y = [], []
  n = past_hours
  pw = prediction_window
  for i in range(n, len(scaled_data)-pw):
      X.append(scaled_data[i-n:i, :])  # Use predictor variables for each timestep
      y.append(scaled_data[i:i+pw, 0])  # X0 is the target variable
  X_train = np.array(X)
  y_train = np.array(y)
  X_train_flattened = X_train#.reshape(X_train.shape[0], -1)
  y_train_flattened = y_train.reshape(y_train.shape[0], -1)
  return X_train_flattened, y_train_flattened

In [ ]:
combinaciones_anteriores = []

In [ ]:
gsearch_path = os.path.join(folder_path, f'investigacion/grid_search/GS_LMST_m_deep_3F_W.txt')
with open(gsearch_path,'r') as fp:
    for line in fp:
      combinaciones_anteriores.append(json.loads(line))

In [ ]:
gsearch_path = os.path.join(folder_path, f'investigacion/grid_search/GS_LMST_m_deep_3F_VA.txt')
with open(gsearch_path,'r') as fp:
    for line in fp:
      combinaciones_anteriores.append(json.loads(line))

In [ ]:
gsearch_path = os.path.join(folder_path, f'investigacion/grid_search/GS_LMST_m_deep_3F_factor.txt')
with open(gsearch_path,'r') as fp:
    for line in fp:
      combinaciones_anteriores.append(json.loads(line))

In [ ]:
combinaciones_anteriores[-2:]

[{'RMSE': 135.99063649129303,
  'Fuente': '3F_W',
  'n_input': 2,
  'n_output': 1,
  'num_neuronas': 50,
  'batch_size': 150,
  'num_layers': 2,
  'MSE': 18493.453213307002},
 {'RMSE': 129.3862796323938,
  'Fuente': '3F_W',
  'n_input': 2,
  'n_output': 1,
  'num_neuronas': 50,
  'batch_size': 50,
  'num_layers': 3,
  'MSE': 16740.809357112004}]

In [ ]:
min_error = 1000000

In [ ]:
string_list = []

In [ ]:
last_result = combinaciones_anteriores[-1]

In [ ]:
for result in combinaciones_anteriores:
  current_string = ''
  for key in result.keys():
    if 'MSE' not in key:
      current_string+=str(result[key])
  string_list.append(current_string)
  if (result['RMSE']<min_error and
    result['n_output']==last_result['n_output'] and
    result['Fuente']==last_result['Fuente']):
    min_error = result['RMSE']

In [ ]:
string_list[-5:]

['3F_W21501501', '3F_W2150502', '3F_W21501002', '3F_W21501502', '3F_W2150503']

In [ ]:
min_error

118.87095272662731

In [ ]:
result.keys()

dict_keys(['RMSE', 'Fuente', 'n_input', 'n_output', 'num_neuronas', 'batch_size', 'num_layers', 'MSE'])

In [ ]:
last_n_output=last_result['n_output']

In [ ]:
last_iset = last_result['Fuente']

In [ ]:
last_min_err = min_error

In [ ]:
last_min_err

118.87095272662731

In [ ]:
for input_set in input_sets.keys():
  df_train_i = df_train[input_sets[input_set]].copy()
  df_val_i = df_val[input_sets[input_set]].copy()
  # scaled data
  scaler.fit(df_train_i)
  scaled_train = scaler.transform(df_train_i)
  scaled_val = scaler.transform(df_val_i)
  # prediction window
  for pw in ventana_prediccion:
    n_output = pw
    if(last_n_output != n_output or input_set != last_iset):
      min_error = 100000 # hay que mejorar
    else:
      min_error = last_min_err

    for fi in factor_ventana_entrada:
      n_input = int(fi*pw)
      if(n_input<1):
        continue
      n_features = len(df_train_i.columns)-1
      for num_neuronas in num_neuronas_set:
        for num_layers in num_layers_set:
          for batch_size in batch_size_set:
            if(str(input_set)+str(n_input)+str(n_output)+str(num_neuronas)+str(batch_size)+str(num_layers)
               in string_list):
              continue
            print("Current options: "+str(input_set)+str(n_input)+str(n_output)+str(num_neuronas)+str(batch_size)+str(num_layers))
            X_train, y_train = get_XY(scaled_train,past_hours=n_input,prediction_window=n_output)
            model = create_Model(n_input,LSTM_neurons=num_neuronas,layers=num_layers,n_features=n_features,n_output=n_output)
            model.fit(X_train,y_train,epochs=10,verbose=False,batch_size=batch_size)

            X_val, y_val = get_XY(scaled_val,past_hours=n_input,prediction_window=n_output)
            predictions = model.predict(X_val,verbose=False)

            flatten_y = y_val.flatten()
            flatten_pred = predictions.flatten()

            true_y = np.zeros(shape=(len(flatten_pred), 1+n_features))
            true_y[:,0] = flatten_y
            true_y = scaler.inverse_transform(true_y)
            true_pred = np.zeros(shape=(len(flatten_pred), 1+n_features))
            true_pred[:,0] = flatten_pred
            true_pred = scaler.inverse_transform(true_pred)
            errorMSE = mean_squared_error(true_y[:,0],true_pred[:,0])
            errorRMSE = np.sqrt(errorMSE)
            if (errorRMSE < min_error):
              min_error = errorRMSE
              last_min_err = min_error
              print('BEST')
              model_path = os.path.join(folder_path, f'investigacion/grid_search/LSTM_deep_model_{input_set}_pw{n_output}.h5')
              model.save(model_path)
            resultado = {
                'RMSE':errorRMSE,
                'Fuente':input_set,
                'n_input':n_input,
                'n_output':n_output,
                'num_neuronas':num_neuronas,
                'batch_size':batch_size,
                'num_layers': num_layers,
                'MSE':errorMSE,
            }
            print(resultado)
            gsearch_path = os.path.join(folder_path, f'investigacion/grid_search/GS_LMST_m_deep_{input_set}.txt')
            with open(gsearch_path,"a") as file:
              resultado = json.dumps(resultado)
              file.write(resultado)
              file.write('\n')


Current options: 3F_W21501003
{'RMSE': 128.3182476999661, 'Fuente': '3F_W', 'n_input': 2, 'n_output': 1, 'num_neuronas': 50, 'batch_size': 100, 'num_layers': 3, 'MSE': 16465.57269278985}
Current options: 3F_W21501503
{'RMSE': 142.75008107768673, 'Fuente': '3F_W', 'n_input': 2, 'n_output': 1, 'num_neuronas': 50, 'batch_size': 150, 'num_layers': 3, 'MSE': 20377.585647686137}
Current options: 3F_W21100501
BEST


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


{'RMSE': 118.3851641486515, 'Fuente': '3F_W', 'n_input': 2, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 50, 'num_layers': 1, 'MSE': 14015.047090503162}
Current options: 3F_W211001001
{'RMSE': 121.89886744012917, 'Fuente': '3F_W', 'n_input': 2, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 100, 'num_layers': 1, 'MSE': 14859.333883186184}
Current options: 3F_W211001501
{'RMSE': 129.6426161907509, 'Fuente': '3F_W', 'n_input': 2, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 150, 'num_layers': 1, 'MSE': 16807.20793278235}
Current options: 3F_W21100502
BEST


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


{'RMSE': 117.46903838564198, 'Fuente': '3F_W', 'n_input': 2, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 50, 'num_layers': 2, 'MSE': 13798.97497924743}
Current options: 3F_W211001002
{'RMSE': 117.95878816572134, 'Fuente': '3F_W', 'n_input': 2, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 100, 'num_layers': 2, 'MSE': 13914.275705525522}
Current options: 3F_W211001502
{'RMSE': 126.78301043751357, 'Fuente': '3F_W', 'n_input': 2, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 150, 'num_layers': 2, 'MSE': 16073.931735598673}
Current options: 3F_W21100503
BEST


...

{'RMSE': 0.03399655998906475, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 50, 'batch_size': 50, 'num_layers': 2, 'MSE': 0.0011557660910900783}
Current options: 3F_factor11501002
{'RMSE': 0.04042229474413895, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 50, 'batch_size': 100, 'num_layers': 2, 'MSE': 0.0016339619123820434}
Current options: 3F_factor11501502
{'RMSE': 0.04047714585418697, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 50, 'batch_size': 150, 'num_layers': 2, 'MSE': 0.0016383993365011256}
Current options: 3F_factor1150503
{'RMSE': 0.037552474159719164, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 50, 'batch_size': 50, 'num_layers': 3, 'MSE': 0.0014101883155163755}
Current options: 3F_factor11501003
{'RMSE': 0.04195902861183811, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 50, 'batch_size': 100, 'num_layers': 3, 'MSE': 0.0017605600820490494}
Current options: 3F_fa

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


{'RMSE': 0.03138614860950985, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 50, 'num_layers': 2, 'MSE': 0.000985090324538237}
Current options: 3F_factor111001002
{'RMSE': 0.04025653282819439, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 100, 'num_layers': 2, 'MSE': 0.0016205884353474927}
Current options: 3F_factor111001502
{'RMSE': 0.04070305562238742, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 150, 'num_layers': 2, 'MSE': 0.0016567387369991642}
Current options: 3F_factor11100503
BEST


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


{'RMSE': 0.02792459514936294, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 50, 'num_layers': 3, 'MSE': 0.0007797830142558241}
Current options: 3F_factor111001003
{'RMSE': 0.04097002644288425, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 100, 'num_layers': 3, 'MSE': 0.0016785430667306347}
Current options: 3F_factor111001503
{'RMSE': 0.04172675311593836, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 100, 'batch_size': 150, 'num_layers': 3, 'MSE': 0.0017411219255984718}
Current options: 3F_factor11150501
{'RMSE': 0.03167055560123914, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 150, 'batch_size': 50, 'num_layers': 1, 'MSE': 0.0010030240920911797}
Current options: 3F_factor111501001
{'RMSE': 0.034094011467901766, 'Fuente': '3F_factor', 'n_input': 1, 'n_output': 1, 'num_neuronas': 150, 'batch_size': 100, 'num_layers': 1, 'MSE': 0.001162401617973417}
Current option

In [ ]:
min_error

In [ ]:
errorRMSE

In [ ]:
y_val.shape

In [ ]:
predictions.shape

In [ ]:
predictions[:2]

In [ ]:
predictions.flatten()[:11]

In [ ]:
len(y_val)

In [ ]:
y_val_narray = np.array(y_val)

In [ ]:
y_val[:2]

In [ ]:
y_val_narray.flatten()[:10]

In [ ]:
y_val[0]

In [ ]:
model.summary()